# Implement standard map-based idioms to filter and display

Steps:

  1. Display the basic map.
  
  2. Read the geocoded shape file.
  
  3. Create markers (to be clustered) for all the businesses in the two zip codes. Create selection for all business and put it on the map.
  
  4. Read the BID shape files.
  
  5. Select the El Cajon BID.
  
  6. Find the businesses within the BID poly.  Create a second clustering for these buisness and put them on the map.
  
Note as you go through these steps the map display is changing.


https://www.sandiego.gov/economic-development/about/bids

In [1]:
import geopandas as gpd
from ipywidgets import HTML, Layout
from ipyleaflet import (Map, Rectangle, GeoJSON,
                        MarkerCluster, GeoData, LayersControl,
                        LayerGroup, Marker, WidgetControl,
                        CircleMarker,
                       basemaps, basemap_to_tiles)

from tqdm import tqdm

In [2]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(32.715, -117.1625), zoom=12,
                  layers=[imagery, osm],
                  layout=Layout(height="700px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())
map_display

Map(center=[32.715, -117.1625], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

# BID Polygons

Ok, this is one of those areas where some design/code would really help!

In [3]:
bids_gdf = gpd.read_file("../data/bids/bids_datasd.shp")

bids_gdf = bids_gdf.to_crs(epsg=4326)

In [4]:
bids_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   objectid   20 non-null     int64   
 1   name       20 non-null     object  
 2   long_name  20 non-null     object  
 3   status     20 non-null     object  
 4   link       20 non-null     object  
 5   geometry   20 non-null     geometry
dtypes: geometry(1), int64(1), object(4)
memory usage: 1.1+ KB


In [5]:
bids_gdf

,objectid,name,long_name,status,link,geometry
0,1,Diamond,Diamond Business Improvement District (BID),Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.11028 32.71086, -117.11043 32.7..."
1,2,Downtown San Diego,Downtown San Diego Business Improvement Distri...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.16400 32.71989, -117.16400 32.7..."
2,3,El Cajon Boulevard Central,El Cajon Boulevard Central Business Improvemen...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.07741 32.75811, -117.07750 32.7..."
3,4,El Cajon Boulevard Gateway,El Cajon Boulevard Gateway Improvement Distric...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.12504 32.75573, -117.12357 32.7..."
4,5,Gaslamp Quarter,Gaslamp Quarter Business Improvement District ...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.15925 32.71452, -117.15923 32.7..."
5,6,Hillcrest,Hillcrest Business Improvement District (BID),Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.14633 32.75046, -117.14609 32.7..."
6,7,La Jolla,La Jolla Business Improvement District (BID),Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.27174 32.85039, -117.27173 32.8..."
7,8,Little Italy,Little Italy Business Improvement District-Pro...,Adopted,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.17451 32.72612, -117.17452 32.7..."
8,9,Mission Hills,Mission Hills Business Improvement District (BID),Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.17783 32.74000, -117.17780 32.7..."
9,10,Morena,Morena Business Improvement District-Proposed,Adopted,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.20633 32.79070, -117.20604 32.7..."


In [6]:
bids = GeoData(geo_dataframe = bids_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'BIDs')

#map_display.add_layer(geo_data)
map_display += bids

In [7]:
bid_html = HTML('''Hover over a district''')
bid_html.layout.margin = '0px 20px 20px 20 px'
bid_control = WidgetControl(widget=bid_html, position='bottomright')

def update_bid_html(feature, **kwargs):
    bid_html.value = f"{feature['properties']['name']}"
    
map_display.add_control(bid_control)  # does += work for this?

bids.on_hover(update_bid_html)

## Go back and look at the map.

Extra credit if you know how display the map so you don't have to go back and forth!

##  All businesses in 92115 and 92116

In [8]:
my_gdf = gpd.read_file("../data/ecb.shp").reset_index()

In [9]:
len(my_gdf)

2355

In [10]:
my_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2355 entries, 0 to 2354
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   index       2355 non-null   int64   
 1   BUSINESS A  2355 non-null   int64   
 2   DBA NAME    2355 non-null   object  
 3   OWNERSHIP   2355 non-null   object  
 4   ADDRESS     2355 non-null   object  
 5   CITY        2355 non-null   object  
 6   ZIP         2355 non-null   object  
 7   STATE       2355 non-null   object  
 8   BUSINESS P  2079 non-null   object  
 9   OWNER NAME  2355 non-null   object  
 10  CREATION D  2355 non-null   object  
 11  START DT    2355 non-null   object  
 12  EXP DT      2355 non-null   object  
 13  NAICS       2355 non-null   int64   
 14  ACTIVITY D  2355 non-null   object  
 15  today       2355 non-null   object  
 16  years       2355 non-null   int64   
 17  naics_code  2355 non-null   int64   
 18  NAICS Desc  2355 non-null   object  
 19

Don't forget that Nominatim doesn't always give you a Point!

In [11]:
valid_geo_gdf = my_gdf.dropna(subset=['geometry']).reset_index()

In [12]:
len(my_gdf) - len(valid_geo_gdf)

18

## Build the markers and put them on the map

In [13]:
business = list()
for i, r in tqdm(valid_geo_gdf.iterrows()):
    marker = CircleMarker(location=(r.geometry.y, r.geometry.x), radius=5, stroke=False, fill_color="blue", fill_opacity=1.0)
    msg = HTML()
    msg.value = f"{r['DBA NAME']}"
    marker.popup = msg
    business.append(marker)
    valid_geo_gdf.iloc[i]['marker'] = marker
valid_geo_gdf['marker'] = business

0it [00:00, ?it/s]<ipython-input-13-3f01d2771fcf>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_geo_gdf.iloc[i]['marker'] = marker
/home/mcmorgan/anaconda3/envs/sherpa/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
2337it [00:14, 159.03it/s]


In [14]:
business_cluster = MarkerCluster(markers=business, name='Businesses')
map_display.add_layer(business_cluster)

#map_display.add_control(LayersControl())

## Need the El Cajon BID geometry

Can get this from the bids_gdf

In [17]:
bids_gdf

,objectid,name,long_name,status,link,geometry
0,1,Diamond,Diamond Business Improvement District (BID),Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.11028 32.71086, -117.11043 32.7..."
1,2,Downtown San Diego,Downtown San Diego Business Improvement Distri...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.16400 32.71989, -117.16400 32.7..."
2,3,El Cajon Boulevard Central,El Cajon Boulevard Central Business Improvemen...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.07741 32.75811, -117.07750 32.7..."
3,4,El Cajon Boulevard Gateway,El Cajon Boulevard Gateway Improvement Distric...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.12504 32.75573, -117.12357 32.7..."
4,5,Gaslamp Quarter,Gaslamp Quarter Business Improvement District ...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.15925 32.71452, -117.15923 32.7..."
5,6,Hillcrest,Hillcrest Business Improvement District (BID),Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.14633 32.75046, -117.14609 32.7..."
6,7,La Jolla,La Jolla Business Improvement District (BID),Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.27174 32.85039, -117.27173 32.8..."
7,8,Little Italy,Little Italy Business Improvement District-Pro...,Adopted,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.17451 32.72612, -117.17452 32.7..."
8,9,Mission Hills,Mission Hills Business Improvement District (BID),Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.17783 32.74000, -117.17780 32.7..."
9,10,Morena,Morena Business Improvement District-Proposed,Adopted,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.20633 32.79070, -117.20604 32.7..."


In [18]:
#i.e.
ecb_bid = bids_gdf.query(f"name == 'El Cajon Boulevard Central'").reset_index()

In [19]:
ecb_bid

,index,objectid,name,long_name,status,link,geometry
0,2,3,El Cajon Boulevard Central,El Cajon Boulevard Central Business Improvemen...,Existing,http://www.sandiego.gov/economic-development/b...,"POLYGON ((-117.07741 32.75811, -117.07750 32.7..."


## Filter based on the polygon

In [20]:
ecb_biz_gdf = valid_geo_gdf[valid_geo_gdf.geometry.within(ecb_bid.iloc[0].geometry)]

In [21]:
len(ecb_biz_gdf)

192

In [22]:
ecb_biz_gdf.columns

Index(['level_0', 'index', 'BUSINESS A', 'DBA NAME', 'OWNERSHIP', 'ADDRESS',
       'CITY', 'ZIP', 'STATE', 'BUSINESS P', 'OWNER NAME', 'CREATION D',
       'START DT', 'EXP DT', 'NAICS', 'ACTIVITY D', 'today', 'years',
       'naics_code', 'NAICS Desc', 'sector', 'sector_des', 'zip_code',
       'geometry', 'marker'],
      dtype='object')

In [23]:
ecb_biz_gdf = ecb_biz_gdf.drop(columns=['level_0']).reset_index()

## Create the second set up businesses for the map

In [24]:
ecb_business = list()
for i, r in tqdm(ecb_biz_gdf.iterrows()):
    marker = CircleMarker(location=(r.geometry.y, r.geometry.x), radius=5, stroke=False, fill_color="blue", fill_opacity=1.0)
    msg = HTML()
    msg.value = f"{r['DBA NAME']}"
    marker.popup = msg
    ecb_business.append(marker)
    ecb_biz_gdf.iloc[i]['marker'] = marker
ecb_biz_gdf['marker'] = ecb_business

0it [00:00, ?it/s]<ipython-input-24-5032c1f56b95>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecb_biz_gdf.iloc[i]['marker'] = marker
192it [00:01, 166.66it/s]


In [25]:
ecb_business_cluster = MarkerCluster(markers=ecb_business, name='Boulevard Businesses')
map_display.add_layer(ecb_business_cluster)

## Now go back and look at the map again.

## This demonstrates the basics

With this we have basic tools to put businesses on a map, display polygons, spatial filtering, ...  Basics